# Train neural networks with `momics` and `tensorflow`

`momics` provides several useful resources to train neural networks with `tensorflow`. This notebook demonstrates how to train a simple neural network with `momics` and `tensorflow`.

## Connect to the data repository

We will tap into the repository generated in the [previous tutorial](integrating-multiomics.ipynb). 

In [1]:
from momics.momics import Momics

## Creating repository
repo = Momics("yeast_CNN_data.momics")

## Check that sequence and some tracks are registered
repo.seq()
repo.tracks()

momics :: INFO :: 2025-03-26 17:19:57,464 :: No cloud config found for momics.Consider populating `~/.momics.ini` file with configuration settings for cloud access.


,idx,label,path
0,0,atac,/home/jaseriza/repos/momics/data/S288c_atac.bw
1,1,scc1,/home/jaseriza/repos/momics/data/S288c_scc1.bw
2,2,mnase,/home/jaseriza/repos/momics/data/S288c_mnase.bw
3,3,atac_rescaled,tmp62h9fu65
4,4,mnase_rescaled,tmp736pjxgo


## Modify some tracks

We can first pre-process the tracks to normalize them, and save them back to the local repository.

In [ ]:
import numpy as np

for track in ["atac", "mnase"]:
    cov = repo.tracks(track)
    # Compute genome-wide 99th percentile
    q99 = np.nanpercentile(np.concatenate(list(cov.values())), 99)
    for chrom in cov.keys():
        arr = cov[chrom]
        # Truncate to genome-wide 99th percentile
        arr = np.minimum(arr, q99)
        # Rescale to [0, 1]
        arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        # Convert NaNs to 0
        arr = np.nan_to_num(arr, nan=0)
        # Store back
        cov[chrom] = arr
    repo.ingest_track(cov, track + "_rescaled")

repo.tracks()

## Define datasets and model 

We will define a simple convolutional neural network with `tensorflow` to predict the target variable `ATAC` from the feature variable `MNase`. This requires to first define a set of genomic coordinates to extract genomic data from. We will use `MNase_rescaled` coverage scores over tiling genomic windows (`features_size` of `1025`, with a stride of `48`) as feature variables to predict `ATAC_rescaled` coverage scores over the same tiling genomic windows, but narrowed down to the a `target_size` of `24` bp around the center of the window. We can split the data into training, testing and validation sets, using `momics.utils.split_ranges()`.

In [2]:
import momics.utils as mutils

# Fetch data from the momics repository
features = "mnase_rescaled"
target = "atac_rescaled"
features_size = 1024 + 1
stride = 48

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins = bins.subset(lambda x: x.Chromosome != "XVI")
bins_split, bins_test = mutils.split_ranges(bins, 0.8)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8)
bins_train

,Chromosome,Start,End
0,I,19632,20657
1,I,193680,194705
2,I,215904,216929
3,I,219744,220769
4,I,14064,15089
...,...,...,...
149233,XV,452256,453281
149234,XV,735024,736049
149235,XV,266496,267521
149236,XV,944208,945233


We now need to define different datasets, for training, testing and validation. We will use `momics.dataset.MomicsDataset()` constructor, indicating the batch size we wish to use in the training process.

In [6]:
from momics.dataset import MomicsDataset

target_size = 24
batch_size = 1000

train_dataset = (
    MomicsDataset(repo, bins_train, features, target, target_size=target_size, batch_size=batch_size).prefetch(2).repeat()
)
val_dataset = MomicsDataset(repo, bins_val, features, target, target_size=target_size, batch_size=batch_size).repeat()
test_dataset = MomicsDataset(repo, bins_test, features, target, target_size=target_size, batch_size=batch_size)
train_dataset

<_RepeatDataset element_spec=((TensorSpec(shape=(None, 1025, 1), dtype=tf.float32, name=None),), (TensorSpec(shape=(None, 24, 1), dtype=tf.float32, name=None),))>

Now is time to define the model architecture. In this example, we will use a simple convolutional neural network (`ChromNN`), pre-defined in `momics.chromnn`. We can instantiate the model, and compile it with the desired optimizer, loss function and metrics.

In [7]:
from momics.chromnn import ChromNN
import tensorflow as tf
from tensorflow.keras import layers  # type: ignore

model = ChromNN(
    layers.Input(shape=(features_size, 1)),
    layers.Dense(target_size, activation="linear"),
).model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1025, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1025, 32)       │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 513, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 513, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 513, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 513, 32)        │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 257, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 257, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 257, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 257, 32)        │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 129, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 129, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 129, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 129, 32)        │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 65, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 65, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 65, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 65, 32)         │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 33, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 33, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1056)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │        25,368 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 40,664 (158.84 KB)

 Trainable params: 40,344 (157.59 KB)

 Non-trainable params: 320 (1.25 KB)

## Fit the model 

Now that we have the datasets and the model, we can fit the model to the training data, using the `fit()` method of the model. We can also evaluate the model on the testing and validation datasets.

In [8]:
import numpy as np
from pathlib import Path
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # type: ignore

callbacks_list = [
    CSVLogger(Path(".chromnn", "epoch_data.csv")),
    ModelCheckpoint(filepath=Path(".chromnn", "Checkpoint.keras"), monitor="val_loss", save_best_only=True),
    EarlyStopping(monitor="val_loss", patience=10, min_delta=1e-5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=6 // 2, min_lr=0.1 * 0.001),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks_list,
    steps_per_epoch=int(np.floor(len(bins_train) // batch_size)),
    validation_steps=int(np.floor(len(bins_val) // batch_size)),
)

Epoch 1/30


/home/jaseriza/micromamba/envs/momics/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1
Received: inputs=('Tensor(shape=(None, 1025, 1))',)
  warnings.warn(msg)
I0000 00:00:1743006061.781632 2751107 service.cc:148] XLA service 0x77b0a80162f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743006061.781667 2751107 service.cc:156]   StreamExecutor device (0): NVIDIA RTX A2000 12GB, Compute Capability 8.6
2025-03-26 17:21:01.876802: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743006062.212015 2751107 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-26 17:21:03.176477: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in 

  3/149 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 1.9127  

I0000 00:00:1743006070.088219 2751107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.5565

/home/jaseriza/micromamba/envs/momics/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1
Received: inputs=('Tensor(shape=(None, 1025, 1))',)
  warnings.warn(msg)


149/149 ━━━━━━━━━━━━━━━━━━━━ 22s 81ms/step - loss: 0.5544 - val_loss: 0.0590 - learning_rate: 0.0010
Epoch 2/30


2025-03-26 17:21:23.382670: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3719', 192 bytes spill stores, 192 bytes spill loads



149/149 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - loss: 0.0511 - val_loss: 0.0422 - learning_rate: 0.0010
Epoch 3/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - loss: 0.0388 - val_loss: 0.0393 - learning_rate: 0.0010
Epoch 4/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 86ms/step - loss: 0.0343 - val_loss: 0.0575 - learning_rate: 0.0010
Epoch 5/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - loss: 0.0321 - val_loss: 0.0495 - learning_rate: 0.0010
Epoch 6/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 12s 82ms/step - loss: 0.0307 - val_loss: 0.0417 - learning_rate: 0.0010
Epoch 7/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - loss: 0.0264 - val_loss: 0.0426 - learning_rate: 1.0000e-04
Epoch 8/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - loss: 0.0261 - val_loss: 0.0420 - learning_rate: 1.0000e-04
Epoch 9/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 86ms/step - loss: 0.0260 - val_loss: 0.0423 - learning_rate: 1.0000e-04
Epoch 10/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 13s 88ms/step - loss: 0.0262 - val_loss: 0.0423 - learning_rat

## Evaluate and save model 

Now let's see how the trained model performs, and save it to the local repository.

In [ ]:
# Evaluate the model
model.evaluate(test_dataset)

## Use the model to predict ATAC-seq coverage

We can now use our trained model to predict ATAC-seq coverage from MNase-seq coverage, for example on a chromosome which has not been used for training.

In [9]:
from momics.query import MomicsQuery
from momics.aggregate import aggregate

## Define tiling 1025 bp windows, with a stride of 1 bp, and extract MNase data from it.
bb = repo.bins(width=features_size, stride=1, cut_last_bin_out=True)["XVI"]
dat = MomicsQuery(repo, bb).query_tracks(tracks=["mnase_rescaled"])
dat = np.array(list(dat.coverage["mnase_rescaled"].values()))

## Now predict the ATAC signal from the MNase signal
predictions = model.predict(dat)

## Export predictions as a bigwig
bb2 = bb.copy()
bb2.Start = bb2.Start + features_size // 2 - target_size // 2
bb2.End = bb2.Start + target_size
chrom_sizes = {chrom: length for chrom, length in zip(repo.chroms().chrom, repo.chroms().length)}
keys = [f"{chrom}:{start}-{end}" for chrom, start, end in zip(bb2.Chromosome, bb2.Start, bb2.End)]
res = {"atac": {k: None for k in keys}}
for i, key in enumerate(keys):
    res["atac"][key] = predictions[i]

aggregate(res, bb2, chrom_sizes, type="mean", prefix="prediction")

29596/29596 ━━━━━━━━━━━━━━━━━━━━ 24s 802us/step


momics :: INFO :: 2025-03-26 17:30:38,488 :: Saved coverage for atac to prediction_atac.bw


{'atac': {'I': array([0., 0., 0., ..., 0., 0., 0.]),
  'II': array([0., 0., 0., ..., 0., 0., 0.]),
  'III': array([0., 0., 0., ..., 0., 0., 0.]),
  'IV': array([0., 0., 0., ..., 0., 0., 0.]),
  'V': array([0., 0., 0., ..., 0., 0., 0.]),
  'VI': array([0., 0., 0., ..., 0., 0., 0.]),
  'VII': array([0., 0., 0., ..., 0., 0., 0.]),
  'VIII': array([0., 0., 0., ..., 0., 0., 0.]),
  'IX': array([0., 0., 0., ..., 0., 0., 0.]),
  'X': array([0., 0., 0., ..., 0., 0., 0.]),
  'XI': array([0., 0., 0., ..., 0., 0., 0.]),
  'XII': array([0., 0., 0., ..., 0., 0., 0.]),
  'XIII': array([0., 0., 0., ..., 0., 0., 0.]),
  'XIV': array([0., 0., 0., ..., 0., 0., 0.]),
  'XV': array([0., 0., 0., ..., 0., 0., 0.]),
  'XVI': array([0., 0., 0., ..., 0., 0., 0.]),
  'Mito': array([0., 0., 0., ..., 0., 0., 0.])}}

This generates a new `bw` file with ATAC-seq coverage over chr16, predicted from MNase-seq coverage.

## Use other feature variables for prediction

We can use other predictor variable, for example the `nucleotide` sequence of tiling genomic windows, to predict `ATAC_rescaled` coverage scores. We can use the same procedure as above, but with a different set of genomic coordinates and feature variables.

In [ ]:
import numpy as np
import momics.utils as mutils

for track in ["ATAC", "MNase"]:
    cov = repo.tracks(track)
    q99 = np.nanpercentile(np.concatenate(list(cov.values())), 99)
    for chrom in cov.keys():
        arr = cov[chrom]
        arr = np.minimum(arr, q99)
        arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        cov[chrom] = np.nan_to_num(arr, nan=0)
    repo.ingest_track(cov, track + "_rescaled")

repo.tracks()


# Fetch data from the momics repository
features = "MNase_rescaled"
target = "ATAC_rescaled"
features_size = 1024 + 1
target_size = 24
stride = 48

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins = bins.subset(lambda x: x.Chromosome != "XVI")
bins_split, bins_test = mutils.split_ranges(bins, 0.8)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8)